In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
# load data
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
subs = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [77]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [80]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Bisa dilihat, dari informasi diatas, dapat diketahui bahwa ada beberapa **kolom** yang datanya tidak lengkap, seperti `Age`, `Cabin`, dan `Embarked`. Karena `Cabin` memiliki data kosong lebih dari 50%, jadi harus dihilangkan, karena tidak efektif jika dilakukan interpolasi. Selain itu, kolom `nama` dan `PassengerId` juga sepertinya tidak begitu memengaruhi survivabilitas orang, jadi bisa di drop juga.

In [7]:
# jumlah nilai unique
train_data.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

In [22]:
# rate survived women and men
women = train_data.loc[train_data.Sex == 
                      'female']['Survived']
men = train_data.loc[train_data.Sex == 'male']['Survived']
rate_women = sum(women)/len(women)
rate_men = sum(men)/len(men)

print("% of women who survived: {0:.2f}".format(rate_women))
print("% of man who survived: {0:.2f}".format(rate_men))

% of women who survived: 0.74
% of man who survived: 0.19


Dari hasil diatas bisa tahu kalau tingkat selamat **wanita** jauh lebih tinggi (74%) dibandingkan **pria** (19%).

In [107]:
df_train = train_data.copy()
df_test = test_data.copy()

df_train = df_train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1)
df_test = df_test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1)

print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       332 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non

In [108]:
# isi missing value pada data Age dan Embarked
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())
df_train['Embarked'] = df_train['Embarked'].fillna('S')

df_test['Age'] = df_test['Age'].fillna(df_test['Age'].mean())
df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].mean())

In [109]:
# isi data kosong
#df_train['Embarked'] = df_train.fillna(df_train['Embarked'].mode()[0])
#df_train['Age'] = df_train.fillna(df_train['Age'].mode()[0])
#df_test['Fare'] = df_test.fillna(df_test['Fare'].mode()[0])
#df_test['Age'] = df_test.fillna(df_test['Age'].mode()[0])

print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Embarked  418 non

In [110]:
# melakukan klasifikasi umur berdasarkan WHO
df_train.loc[(df_train['Age'] > 0) & (df_train['Age'] <= 1), 'Age' ] = 0 #'balita'
df_train.loc[ (df_train['Age'] > 1) & (df_train['Age'] <= 10), 'Age' ] = 1 #'anak_anak'
df_train.loc[ (df_train['Age'] > 10) & (df_train['Age'] <= 19), 'Age' ] = 2 #'remaja'
df_train.loc[ (df_train['Age'] > 19) & (df_train['Age'] <= 60), 'Age' ] = 3 #'dewasa'
df_train.loc[ (df_train['Age'] > 60), 'Age' ] = 4 #'lanjut_usia'

df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,3.0,1,0,7.2500,S
1,1,1,female,3.0,1,0,71.2833,C
2,1,3,female,3.0,0,0,7.9250,S
3,1,1,female,3.0,1,0,53.1000,S
4,0,3,male,3.0,0,0,8.0500,S


In [111]:
# melakukan klasifikasi umur berdasarkan WHO
df_test.loc[(df_test['Age'] > 0) & (df_test['Age'] <= 1), 'Age' ] = 0 #'balita'
df_test.loc[ (df_test['Age'] > 1) & (df_test['Age'] <= 10), 'Age' ] = 1 #'anak_anak'
df_test.loc[ (df_test['Age'] > 10) & (df_test['Age'] <= 19), 'Age' ] = 2 #'remaja'
df_test.loc[ (df_test['Age'] > 19) & (df_test['Age'] <= 60), 'Age' ] = 3 #'dewasa'
df_test.loc[ (df_test['Age'] > 60), 'Age' ] = 4 #'lanjut_usia'

In [112]:
# convert to categorical
df_train['Sex'] = pd.Categorical(df_train['Sex'])
df_train['Age'] = pd.Categorical(df_train['Age'])
#df_train['Ticket'] = pd.Categorical(df_train['Ticket'])
df_train['Embarked'] = pd.Categorical(df_train['Embarked'])

# update baru
df_train['Pclass'] = pd.Categorical(df_train['Pclass'])
df_train['SibSp'] = pd.Categorical(df_train['SibSp'])
df_train['Parch'] = pd.Categorical(df_train['Parch'])

In [113]:
# convert to categorical
df_test['Sex'] = pd.Categorical(df_test['Sex'])
df_test['Age'] = pd.Categorical(df_test['Age'])
#df_test['Ticket'] = pd.Categorical(df_test['Ticket'])
df_test['Embarked'] = pd.Categorical(df_test['Embarked'])

df_test['Pclass'] = pd.Categorical(df_test['Pclass'])
df_test['SibSp'] = pd.Categorical(df_test['SibSp'])
df_test['Parch'] = pd.Categorical(df_test['Parch'])

In [114]:
# Pembagian data input dan output
X_train = df_train.drop('Survived', axis=1)
y_train = df_train['Survived']

X_test = df_test

print('train shape', X_train.shape)
print('test_shape', X_test.shape)

train shape (891, 7)
test_shape (418, 7)


In [115]:
# one hot encoding
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

print('X train shape', X_train.shape)
print('X test_shape', X_test.shape)

X train shape (891, 28)
X test_shape (418, 29)


Bisa dilihat dari dimensi variabel diatas, kalau dimensi data train dan test tidak sama. Oleh karena itu, selanjutnya perlu dilakukan "penyamaan kolom".

In [116]:
# menyamakan kolom yg ada di data train dengan di data test
dummy_absent = set(X_train.columns) - set(X_test.columns)
for col in dummy_absent:
    X_test[col] = 0

X_test = X_test[X_train.columns]

print('X train shape', X_train.shape)
print('X test_shape', X_test.shape)

X train shape (891, 28)
X test_shape (418, 28)


In [117]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, random_state=1)

In [118]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(model, X_train, y_train, cv=10)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.813 +/- 0.039


In [119]:
# Prediksi terhadap data test
prediksi = model.predict(X_test)

# output
output = pd.DataFrame({'PassengerId':test_data.PassengerId, 'Survived':prediksi})
output.to_csv('RF3_subs.csv', index=False)
print('Output sudah berhasil di save!')

Output sudah berhasil di save!
